In [9]:
import cv2

In [21]:
#カメラから映像を読み取る。（）内の引数はカメラの接続ID ※接続カメラが1台なら0になる。
cap = cv2.VideoCapture(1)

#四角い赤枠の色を指定
red = (0, 0, 255)

#★avgという変数に、比較用のフレームの配列を残す（要調査）
avg = None

while True:
    
    #retの中には映像取得が成功したかどうかが入っている（True or False）
    #frameの中には、カメラから取得した映像データが入っている（RGBの3次元データが入っている）
    ret, frame = cap.read()
    
    #★多分映像取得が失敗したらbreakするという事だと思う
    #これは録画や再生が終了したらループを抜けるという事とのこと
    if not ret:
        break
    
    #★グレースケールに変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #不明（要調査）
    if avg is None:
        avg = gray.copy().astype("float")
        continue
    
    #★フレームの差？（要調査）
    #0.7などの(gray, avg, ●●●の数字を小さくするほど感度が上がる)
    cv2.accumulateWeighted(gray, avg, 0.50)
    frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))
    
    #★不明
    thresh = cv2.threshold(frameDelta, 10, 255, cv2.THRESH_BINARY)[1]
    contours, hierarchy = cv2.findContours(thresh.copy(),
                                           cv2.RETR_EXTERNAL,
                                          cv2.CHAIN_APPROX_SIMPLE)
    #ここのdrawContoursは四角形ではなく輪郭も表示する
    # frame = cv2.drawContours(frame, contours, -1, (0, 255, 0), 3)
    
    #ここのif w < ●●が、多分w＝ワイド＝幅だと思うが、大きいと大きいものしか認識しなくなる？
    for target in contours:
        x, y, w, h = cv2.boundingRect(target)
        if w < 200: continue
        cv2.rectangle(frame, (x, y), (x+w, y+h), red, 2)
    

#     for target in contours:
#         x, y, w, h = cv2.boundingRect(target)
        
#         if w < 10000:
#             continue
        
#         areaframe = cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0),2)
        
        
    #カメラの映像を出力
    cv2.imshow("camera", frame)
    
    #カメラ映像を連続で取得するwhileからESCを押すと抜けるという文
    key = cv2.waitKey(10)
    if key == 27:
        break

cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1